## Alignement des chaînes OCR avec les outils ISRI

In [8]:
import isri_tools

a = "👉 étagères𐌰"
b = "etag.µbleble"

Fonction d'alignement des chaînes.

In [9]:

A, B = isri_tools.align(a, b, 'x')
print(A)
print(B)

👉 étagèrxxes𐌰x
exxtag.µbleble


Pour obtenir la transformation d'alignment.

On obtient ici deux listes de la même taille que les chaînes d'origine et qui donnent les positions dans l'alignement final.

In [10]:
A, B = isri_tools.get_align_map(a, b)
print(A)
print(B)

[0, 1, 2, 3, 4, 5, 6, 7, 10, 11, 12]
[0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]


## Retagging d'une chaîne OCR à partir des Entités Nommées d'une chaîne XML

In [11]:
from ner_sync_ref import add_tags_prediction

In [12]:
A = "<PER>Anthony</PER>, fab. du <ACT>pêche</ACT>, <LOC>Châtelet</LOC>"
B = "Mme Antoine, fab la pecheur du chatelet et du Faub. St Antoine"
add_tags_prediction(A, B)

'Mme <PER>Antoine</PER>, fab la <ACT>peche</ACT>ur du <LOC>chatelet</LOC> et du Faub. St Antoine'

## Calcul de l'accuracy et rapport

In [14]:
a = "👉 étagères𐌰"
b = "etag.µbleble"
stats = isri_tools.compute_accurary_stats(a, b)
stats

UNLV-ISRI OCR Accuracy Report Version 5.1
-----------------------------------------
      11   Characters
      10   Errors
    9.09%  Accuracy

       0   Reject Characters
       0   Suspect Markers
       0   False Marks
    0.00%  Characters Marked
    9.09%  Accuracy After Correction

     Ins    Subst      Del   Errors
       0        0        0        0   Marked
       2        5        3       10   Unmarked
       2        5        3       10   Total

   Count   Missed   %Right
       1        1     0.00   ASCII Spacing Characters
       6        2    66.67   ASCII Lowercase Letters
       2        2     0.00   Latin1 Lowercase Letters
       1        1     0.00   Combining Diacritical Marks
       1        1     0.00   Private Use Area
      11        7    36.36   Total

  Errors   Marked   Correct-Generated
       4        0   {

In [19]:


print("Characters:", stats.characters)
print("Errors:", stats.errors)
print("Accuracy: {:.2f}%".format(100 * (1 - stats.errors / stats.characters)))


Characters: 11
Errors: 10
Accuracy: 9.09%
